In [964]:
""" 
Análisis sobre la relación entre el crecimiento del número de ataques de tiburón y las regiones con mayor
masa de plástico en sus mares.

Este es el ranking de los océanos con mayor contaminación: 
https://ourworldindata.org/grapher/surface-plastic-mass-by-ocean
"""


' \nAnálisis sobre la relación entre el crecimiento del número de ataques de tiburón y las regiones con mayor\nmasa de plástico en sus mares.\n\nEste es el ranking de los océanos con mayor contaminación: \nhttps://ourworldindata.org/grapher/surface-plastic-mass-by-ocean\n'

In [965]:
# Importo pandas para trabajar con dataframes, y numpy para posibles operaciones numéricas con los datos.

import pandas as pd
import numpy as np

In [966]:
# Leo el dataset. Tengo problemas al leerlo porque la codificación es Utt8, incompatible con mi ordenador. 
# Por tanto, cambio la codificación a ISO-8859-1.

dataset = pd.read_csv('GSAF5.csv', encoding = "ISO-8859-1")

In [967]:
# Head para primera visualización de los datos

dataset.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [968]:
# Muestro los nombres de las columnas de mi dataset, para ver si me puedo deshacer de alguna que no 
# tenga relevancia para mi análisis.

print(dataset.columns.values)

['Case Number' 'Date' 'Year' 'Type' 'Country' 'Area' 'Location' 'Activity'
 'Name' 'Sex ' 'Age' 'Injury' 'Fatal (Y/N)' 'Time' 'Species '
 'Investigator or Source' 'pdf' 'href formula' 'href' 'Case Number.1'
 'Case Number.2' 'original order' 'Unnamed: 22' 'Unnamed: 23']


In [969]:
# Hago una lista de las columnas de las que me quiero deshacer. No me deshago de "Case Number" porque lo utilizaré
# después para eliminar duplicados.

unrelevant_cols = ["Date", "Type", "Location", "Activity", "Sex ", "Age", "Injury", "Fatal (Y/N)", 
                   "Time", "Species ",  "Name", "Investigator or Source", "pdf", "href formula", "href", 
                   "Case Number.1", "Case Number.2", "original order", "Unnamed: 22", "Unnamed: 23"]

In [970]:
# Me deshago de ellas con un drop.

dataset = dataset.drop(unrelevant_cols, axis=1)

In [971]:
# Muestro el head de mi dataset actualizado.

display(dataset.head(8))

,Case Number,Year,Country,Area
0,2016.09.18.c,2016,USA,Florida
1,2016.09.18.b,2016,USA,Florida
2,2016.09.18.a,2016,USA,Florida
3,2016.09.17,2016,AUSTRALIA,Victoria
4,2016.09.15,2016,AUSTRALIA,Victoria
5,2016.09.15.R,2016,AUSTRALIA,Western Australia
6,2016.09.11,2016,USA,Florida
7,2016.09.07,2016,USA,Hawaii


In [972]:
# Compruebo cuántos valores nulos tengo en cada columna. 

null_cols = dataset.isnull().sum()

null_cols[null_cols > 0]

Country     43
Area       402
dtype: int64

In [973]:
# Si los registros cuyo valor en Country es null no tienen valor null en Area, puedo detectar a qué océano 
# pertenece el registro. Sin embargo, si la mayoría de los resgistros cuyo valor en Country tienen valor null
# en Area también, me puedo deshacer directamente de los resgistros cuyo valor en Country es null, para simplificar.

# Miro a ver cuántos registros que tengan valor null en Country, lo tienen también en Area.


"""
null_countries = dataset[dataset.Country.isnull() & dataset.Country.isnull()]

len(null_countries.index)

"""

# He intentado eso pero me sale length = 43, lo que significaría que todos los registros con Country = null, tienen 
# Area = null también. Eso no es así, ya que:

display(dataset[dataset.Country.isnull()].head())

# Por tanto, pruebo esto ahora

null_countries = dataset[dataset.Country.isnull()]
null_countries_area = null_countries[null_countries.Area.isnull()]

len(null_countries_area.index)

# Esto sí que funciona

,Case Number,Year,Country,Area
303,2014.08.00,2014,NaN,NaN
2731,1983.00.00.d,1983,NaN,English Channel
3153,1970.11.00,1970,NaN,NaN
3162,1970.08.02,1970,NaN,Caribbean Sea
3163,1970.07.05,1970,NaN,NaN


30

In [974]:
# Para 43 registros cuyo valor en Country es null, 30 son null también en Area (el 70%). Por tanto, byebye.

dataset = dataset[dataset.Country.notnull()]

In [975]:
# Voy a tener que usar los años como enteros para medir el paso en el tiempo, así que compruebo que el type de Year 
# sea integer:

dataset.dtypes

Case Number    object
Year            int64
Country        object
Area           object
dtype: object

In [976]:
# Para poder categorizar los países, tengo que limpiar los strings de caracteres especiales. Para demostrar que sé
# crear nuevas columnas y que sé cambiar su nombre, creo columnas nuevas en las que aparezcan los mismos valores 
# pero limpios. (Realmente lo hago porque no sé limpiar directamente los caracteres de una columna jeje). 
# Ya he aprendido. 

# Primero cambio todos los espacios por _, para no cargarme los separadores de strings:

dataset['Country'] = dataset['Country'].str.replace(' ', '_')
dataset['Area'] = dataset['Area'].str.replace(' ', '_')

In [977]:
# Y ahora borro todos los caracteres especiales:

dataset['Country'] = dataset['Country'].str.replace('\W', '')
dataset['Area'] = dataset['Area'].str.replace('\W', '')

In [ ]:
# Tengo que convertir todos los valores de Area a string

for Area in dataset:
    for r in dataset[Area]:
        rs = str(r)
        dataset['Area'] = dataset['Area'].str.replace(r, rs)

In [ ]:
# Si había algún espacio al final o al principio, tengo que deshacerme ahora de los "_" del final/principio:

for Country in dataset:
    for r in dataset["Country"]:
        if r[-1] == "_":
            r_no_ = r.replace(r, r[:-2])
            dataset['Country'] = dataset['Country'].str.replace(r, r_no_)
        if r[0] == "_":
            r_no__ = r.replace(r, r[1:])
            dataset['Country'] = dataset['Country'].str.replace(r, r_no__)
            
for Area in dataset:
    for r in dataset["Area"]:
        if r[-1] == "_":
            r_no_ = r.replace(r, r[:-2])
            dataset['Area'] = dataset['Area'].str.replace(r, r_no_)
        if r[0] == "_":
            r_no__ = r.replace(r, r[1:])
            dataset['Area'] = dataset['Area'].str.replace(r, r_no__)

In [ ]:
# Si había dos espacios seguidos entre los strings de los países, tengo que cambiar "__" por "_":

for Country in dataset:
    for r in dataset["Country"]:
        if "__" in r:
            rno__ = r.replace("__", "_")
            dataset['Country'] = dataset['Country'].str.replace(r, rno__ )
            
for Country in dataset:
    for r in dataset["Areas"]:
        if "__" in r:
            rno__ = r.replace("__", "_")
            dataset['Areas'] = dataset['Areas'].str.replace(r, rno__ )

In [ ]:
# Ahora tengo que crear un diccionario para relacionar cada país con el/los océanos que costea.
# Hago lista de los países

list_countries = []

for countries in dataset["Country"]:
    list_countries.append(countries)
    

In [ ]:
# Como no quiero que los países se repitan, convierto la lista en una tupla:

tup_countries = set(list_countries)

print(len(tup_countries))
print(tup_countries)

In [ ]:
# Para simplificar mi diccionario, me voy a deshacer de todos aquellos registros cuyos países no estén en el top 50

dataset["Country"].value_counts()

In [ ]:
# Para ello, voy a crear una columna "Counts" que indique el count del país de cada registro:

dataset['Counts'] = dataset.groupby(['Country'])['Area'].transform('count')

In [ ]:
# Y ahora voy a eliminar todos los registros cuyo valor de Counts sea menor que 20

dataset = dataset[dataset.Counts >= 35]

In [ ]:
# Ya no me sirve de nada la columna Counts, así que me la bajo también

dataset = dataset.drop("Counts", axis = 1)

In [ ]:
display(dataset)

In [ ]:
dataset["Area"].value_counts()